## Okta の概要

Okta は、企業向けに安全な ID ソリューションを提供するクラウドベースの ID およびアクセス管理サービスであり、アプリケーションやサービス全体でシームレスな認証と認可を可能にします。

主な機能:
* **シングルサインオン（SSO）** - ユーザーは一度認証するだけで複数のアプリケーションにアクセス可能
* **多要素認証（MFA）** - 追加の検証方法によるセキュリティ強化
* **アダプティブ認証** - ユーザーの行動とコンテキストに基づくリスクベースの認証ポリシー
* **ユニバーサルディレクトリ** - 一元化されたユーザー管理とプロファイル同期
* **API アクセス管理** - API セキュリティのための OAuth 2.0 と OpenID Connect のサポート

## 学習目標
Okta は AgentCore Identity の ID プロバイダーとして使用でき、ユーザーを認証し、ユーザーに代わって保護されたリソースにアクセスするようエージェントを認可させることができます。このノートブックでは、Inbound 認証に Okta を使用する方法を学びます - エージェントを呼び出す前にユーザーを認証します。

## 認証コードフロー
OAuth 2.0 認証コードフローは、Web アプリケーションがユーザーを安全に認証し、アクセストークンを取得するための推奨アプローチです。このフローには以下が含まれます：
1. ユーザーを認証のために Okta にリダイレクト
2. ログイン成功後に認証コードを受信
3. コードをアクセストークンとリフレッシュトークンに交換
4. トークンを使用して保護されたリソースにアクセス

この統合パターンにより、AgentCore はアプリケーションの安全で標準ベースの認証を維持しながら、Okta の堅牢な ID 管理機能を活用できます。

## チュートリアルアーキテクチャ

```
┌──────────┐  1. 認証情報  ┌──────────┐  2. JWT トークン  ┌──────────┐
│  クライアント  │ ───────────────► │   Okta   │ ─────────────► │  クライアント  │
└──────────┘                  └──────────┘                └──────────┘
                                                                 │
                                                                 │ 3. Bearer JWT
                                                                 ▼
┌──────────┐                  ┌──────────┐                ┌──────────┐
│  クライアント  │ ◄─────────────── │ Bedrock  │ ─────────────► │  エージェント   │
└──────────┘  6. レスポンス    │AgentCore │  4. 呼び出し    └──────────┘
                              └──────────┘                      │
                                    ▲                           │
                                    └─────── 5. レスポンス ───────┘
```

<figure>
    <img src="images/16.png">
</figure>

## チュートリアルの詳細

| 情報              | 詳細                                                                          |
|:-----------------|:------------------------------------------------------------------------------|
| チュートリアルタイプ | 会話型                                                                       |
| エージェントタイプ   | シングル                                                                      |
| エージェントフレームワーク | Strands Agents                                                             |
| LLM モデル        | Anthropic Claude Sonnet 3.5                                                  |
| チュートリアルコンポーネント | AgentCore Runtime でエージェントをホスト。Strands Agent と Amazon Bedrock Model を使用 |
| チュートリアル分野  | クロスバーティカル                                                             |
| 例の複雑さ        | 簡単                                                                          |
| Inbound Auth     | Okta                                                                          |
| 使用する SDK      | Amazon BedrockAgentCore Python SDK と boto3                                   |

### 主な機能

* Okta を使用した Inbound Auth で Amazon Bedrock AgentCore Runtime にエージェントをホスト
* Amazon Bedrock モデルを使用
* Strands Agents を使用

## 前提条件

このチュートリアルを実行するには以下が必要です：
* Python 3.10+
* 新しい IAM ロール、ポリシー、ユーザーを作成する IAM 権限
* 新しい AgentCore Agent を作成する IAM 権限
* Okta アカウント
* Amazon Bedrock AgentCore SDK
* Strands Agents
* Docker が実行中であること

## 学習目標 1: AgentCore で使用するための Okta のセットアップ

## ステップ 1: Okta の IDP をセットアップする

AgentCore Runtime を Okta 認証で設定する前に、Okta を ID プロバイダーとしてセットアップする必要があります。このセクションでは、Okta テナントの作成、アプリケーションの設定、必要なユーザーとクレームのセットアップについて説明します。

### 1.1 Okta 開発者アカウントを作成する

まだ Okta アカウントをお持ちでない場合は、https://developer.okta.com/signup/ にアクセスし、「Sign up for Integrator Free Plan」を選択してサインアップします。

### 1.2 テストユーザーを追加する

1. Okta アカウントにログインします。
2. **Directory** を選択し、**People** をクリックして **Add person** をクリックします。

   <figure>
       <img src="images/9.png">
   </figure>

3. フォームに入力します：
   - **Activation** で **Activate now** を選択します。
   - **I will set password** にチェックを入れ、ユーザーのパスワードを設定します。
   - **User must change password on first login** のチェックを外します。
   - **Save** をクリックします。

   <figure>
       <img src="images/10.png">
   </figure>

### 1.3 アプリケーション統合を作成する

4. **Applications** を選択し、**Create App Integration** をクリックします。

   <figure>
       <img src="images/1.png">
   </figure>

5. サインイン方法として **OIDC - OpenID Connect** を選択し、アプリケーションタイプとして **Web Application** を選択します。

   <figure>
       <img src="images/2_enhanced.png">
   </figure>

6. アプリケーションを設定します：
   - アプリ統合名に **AgentCore Inbound Auth** と入力します
   - 付与タイプとして **Authorization Code** を選択します。
   - エージェントを実行するリージョンに応じて、リダイレクト URL として「https://bedrock-agentcore.us-west-2.amazonaws.com/identities/oauth2/callback」または「https://bedrock-agentcore.us-east-1.amazonaws.com/identities/oauth2/callback」を使用します。

   <figure>
       <img src="images/3_enhanced.png">
   </figure>

   - 割り当てで **Allow everyone in your organization to access** を選択し、**Enable immediate access** にチェックを入れたままにします。**Save** をクリックします。

   <figure>
       <img src="images/5_enhanced.png">
   </figure>

   - 後で使用するために **Client ID** と **Secret** をコピーします。

   <figure>
       <img src="images/6_enhanced.png">
   </figure>

### 1.4 認可サーバーを設定する

7. 左側のメニューで **Security** を選択し、**API** をクリックして、認可サーバーの名前をクリックします。

   <figure>
       <img src="images/7_enhanced.png">
   </figure>

   - **Audience** をコピーして後で使用するために保存します。
     > **注意**：この例ではデフォルトの **Audience** が変更されています。他のアプリに影響を与えないように、audience を変更する予定がある場合は新しい認可サーバーを追加することをお勧めします。
   
   - **Scopes** をクリックし、**agentcore** という名前の新しいスコープを追加します。

   <figure>
       <img src="images/agencore.png">
   </figure>

   - **Claims** をクリックし、以下の **client_id** と **scope** クレームを追加します。

   <figure>
       <img src="images/8.png">
   </figure>

### 1.5 設定値を収集する

Okta のセットアップが完了したら、以下の値を持っているはずです：

- **OKTA_CLIENT_ID**: General タブからのアプリケーションの Client ID
- **OKTA_CLIENT_SECRET**: General タブからのアプリケーションの Client Secret  
- **OKTA_AUDIENCE**: Audience（例：`testagentcore`）
- **OKTA_TOKEN_URL**: Okta ドメイン + `/oauth2/default/v1/token`
- **OKTA_DISCOVERY_URL**: Okta ドメイン + `/oauth2/default/.well-known/openid-configuration`

次のステップで必要になるので、これらの値を手元に置いておいてください。

注意:
1. Okta は AWS のサービスではありません。Okta に関連するコストについては、Okta のドキュメントを参照してください。
2. 以下のステップで使用されているスクリーンショットは変更される可能性があります。Okta アプリケーションのセットアップに関する最新のガイダンスについては、Okta のドキュメントを参照することをお勧めします。

## 学習目標 2 - Inbound 認証に Okta を使用したシンプルなエージェントのセットアップ

#### 前提条件
1. 必要なパッケージをインストール
2. パッケージをインポート
3. ノートブック全体で使用するアカウント ID を取得
4. AWS リージョンを「us-west-2」に設定。Bedrock AgentCore をサポートする任意のリージョンを使用できます。https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/agentcore-regions.html を参照してください

## ステップ 2: 環境のセットアップ

まず、開発環境をセットアップし、必要な依存関係をインストールしましょう。

In [ ]:
# Create and activate virtual environment
!python -m venv .venv
!source .venv/bin/activate

このコードを実行するには、Strands Agents モジュールを Python 環境にインストールする必要があります。

Strands Agents モジュール、AgentCore SDK、AgentCore starter toolkit を依存関係ファイルに追加し、**requirements.txt** として保存します：

In [ ]:
%%writefile requirements.txt
strands-agents
strands-agents-tools
bedrock-agentcore
bedrock-agentcore-starter-toolkit
PyJWT

In [ ]:
# Install required packages from requirements.txt
!pip install --force-reinstall -U -r requirements.txt --quiet

In [ ]:
# Verify that all required packages are installed correctly
try:
    import bedrock_agentcore
    import strands
    print("✅ All packages installed successfully")
    print("✅ bedrock-agentcore: imported")
    print("✅ strands: imported")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("Please ensure all packages are installed correctly")

## ステップ 3: 環境変数の設定


ノートブック全体で必要な主要情報の環境変数を設定します。

In [ ]:
# Okta Configuration - Replace with your values
import os

os.environ['OKTA_CLIENT_ID'] = 'YOUR_CLIENT_ID_VALUE'
os.environ['OKTA_CLIENT_SECRET'] = 'YOUR_CLIENT_SECRET_VALUE'
os.environ['OKTA_AUDIENCE'] = 'YOUR_OKTA_AUDIENCE'
os.environ['OKTA_TOKEN_URL'] = 'https://your.okta.com/oauth2/default/v1/token'
os.environ['OKTA_DISCOVERY_URL'] = 'https://your.okta.com/oauth2/default/.well-known/openid-configuration'

## ステップ 4: エージェントコード
このノートブックの主な学習目標は Okta を使用した Inbound 認証を学ぶことなので、エージェントはシンプルにしています

In [ ]:
%%writefile simple_agent.py
import argparse, json
from strands import Agent, tool
from bedrock_agentcore.runtime import BedrockAgentCoreApp

app = BedrockAgentCoreApp()
agent = Agent()

@app.entrypoint
def invoke(payload):
    """Simple agent function for inbound auth demo"""
    user_message = payload.get("prompt", "Hello! How can I help you today?")
    
    # Get session information if available
    session_id = payload.get("session_id", "no-session")
    
    # Simple response with session awareness
    response = f"Hello! I'm a simple agent with session ID: {session_id}. You asked: {user_message}"
    
    result = agent(response)
    return {"result": result.message}

if __name__ == "__main__":
    app.run()

## ステップ 5: Okta 認証で AgentCore Runtime を設定する

Okta OAuth 認証で AgentCore Runtime を設定します。

In [ ]:
import boto3
from bedrock_agentcore_starter_toolkit import Runtime

# Get account ID and region
sts = boto3.client('sts')
account_id = sts.get_caller_identity()['Account']
region = 'us-west-2'

print(f"Account ID: {account_id}")
print(f"Region: {region}")

# Use environment variables
discovery_url = os.getenv('OKTA_DISCOVERY_URL')
client_id = os.getenv('OKTA_CLIENT_ID')
audience = os.getenv('OKTA_AUDIENCE')

print(f"Discovery URL: {discovery_url}")
print(f"Client ID: {client_id[:4]}****{client_id[-4:] if client_id else 'None'}")  # Masked
print(f"Audience: {audience}")

agentcore_runtime = Runtime()

# Try with OAuth configuration
try:
    response = agentcore_runtime.configure(
        entrypoint="simple_agent.py",
        auto_create_execution_role=True,
        auto_create_ecr=True,
        requirements_file="requirements.txt",
        region=region,
        agent_name="okta_inbound_auth_agent",
        authorizer_configuration={
            "customJWTAuthorizer": {
                "discoveryUrl": discovery_url,
                "allowedClients": [client_id],
                "allowedAudience": [audience]
            }
        }
    )
    print("✅ OAuth configuration successful")
except Exception as e:
    print(f"❌ OAuth configuration failed: {e}")

response

## ステップ 6: エージェントを AgentCore Runtime に起動する

エージェントを設定したので、AgentCore Runtime に起動しましょう。

In [ ]:
launch_result = agentcore_runtime.launch()
launch_result

## ステップ 7: AgentCore Runtime のステータスを確認する

エージェントが準備完了になるまでデプロイステータスを監視します。

In [ ]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

## ステップ 8: テスト用にエージェント ARN を保存する

テスト目的でエージェント ARN を抽出して保存します。

テスト目的でエージェント ARN を抽出して保存します

In [ ]:
# Extract ARN from launch_result
if hasattr(launch_result, 'agent_arn') and launch_result.agent_arn:
    agent_arn = launch_result.agent_arn
    os.environ['AGENT_ARN'] = agent_arn
    print(f"📝 Agent ARN: {agent_arn}")
    print(f"📝 Agent ID: {launch_result.agent_id}")
    print(f"📝 ECR URI: {launch_result.ecr_uri}")
else:
    print("⚠️  Could not extract Agent ARN from launch result")
    print("Launch result:", launch_result)

# Check deployment status
if status == 'READY':
    print("✅ Agent deployed successfully and ready for testing!")
elif status in ['CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']:
    print(f"❌ Agent deployment failed with status: {status}")
else:
    print(f"⚠️  Unexpected status: {status}")

## ステップ 9: テストクライアントを作成する

Okta OAuth フローとセッションサポート付きのエージェント呼び出しを検証するテストクライアントを作成します。

In [ ]:
import requests
import json
import time
import urllib.parse
import logging
import uuid

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Configuration
client_id = os.getenv('OKTA_CLIENT_ID')
client_secret = os.getenv('OKTA_CLIENT_SECRET')
audience = os.getenv('OKTA_AUDIENCE')
token_url = os.getenv('OKTA_TOKEN_URL')
AGENT_ARN = os.getenv('AGENT_ARN')

print("✅ Configuration loaded successfully")

Okta から OAuth アクセストークンを取得する関数を定義します

In [ ]:
def get_oauth_token():
    """Get OAuth token from Okta"""
    data = {
        'grant_type': 'client_credentials',
        'scope': 'agentcore'
    }
    
    logger.info("🔐 Getting OAuth token...")
    
    response = requests.post(
        token_url,
        data=data,
        auth=(client_id, client_secret)
    )
    
    response.raise_for_status()
    token_data = response.json()
    logger.info("✅ OAuth token obtained")
    return token_data['access_token']

認証とセッションサポート付きでエージェントを呼び出す関数を定義します

In [ ]:
def invoke_agent(access_token, query, session_id=None):
    """Invoke agent with session ID support"""
    escaped_agent_arn = urllib.parse.quote(AGENT_ARN, safe='')
    url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_agent_arn}/invocations?qualifier=DEFAULT"
    
    # Generate session ID if not provided
    if not session_id:
        session_id = f'okta-inbound-session-{int(time.time())}-{uuid.uuid4().hex[:8]}'
    
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json',
        'X-Amzn-Bedrock-AgentCore-Runtime-Session-Id': session_id
    }
    
    payload = {
        'prompt': query,
        'session_id': session_id
    }
    
    logger.info(f"🚀 Invoking agent with query: {query}")
    logger.info(f"📋 Session ID: {session_id}")
    
    response = requests.post(url, headers=headers, json=payload, timeout=300)
    response.raise_for_status()
    
    result = response.json()
    logger.info("✅ Agent response received")
    return result, session_id

### 認証なしリクエストのテスト（失敗するはず）

まず、エージェントが認証なしリクエストを適切に拒否することを確認しましょう：

In [ ]:
# Test unauthenticated request - this should fail
print("=" * 50)
print("TEST: Unauthenticated Request (Should Fail)")
print("=" * 50)

try:
    escaped_agent_arn = urllib.parse.quote(AGENT_ARN, safe='')
    url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_agent_arn}/invocations?qualifier=DEFAULT"
    
    headers = {
        'Content-Type': 'application/json'
        # Note: No Authorization header
    }
    
    payload = {'prompt': 'Hello without authentication'}
    
    response = requests.post(url, headers=headers, json=payload, timeout=30)
    print(f"❌ Unexpected success! Status: {response.status_code}")
    print(f"Response: {response.text}")
    
except requests.exceptions.HTTPError as e:
    print(f"✅ Expected authentication failure: {e.response.status_code}")
    print(f"Error message: {e.response.text}")
except Exception as e:
    print(f"✅ Expected authentication error: {e}")

print("\n🔒 This confirms that authentication is required!")

認証されたリクエスト用に Okta から OAuth アクセストークンを取得します

In [ ]:
# Get OAuth token
access_token = get_oauth_token()
print("✅ OAuth token obtained successfully")

セッション ID トラッキング付きで認証されたエージェント呼び出しをテストします

In [ ]:
# Test 1: Simple query with session ID
print("=" * 50)
print("TEST 1: Simple Query with Session ID")
print("=" * 50)

result1, session_id1 = invoke_agent(access_token, "Hello can you guide me about AWS security best practices for authentication?")
print(f"Session ID used: {session_id1}")
print(json.dumps(result1, indent=2))

同じセッション ID を再利用してセッションの継続性をテストします

In [ ]:
# Test 2: Continue conversation with same session ID
print("=" * 50)
print("TEST 2: Continue Conversation with Same Session")
print("=" * 50)

result2, session_id2 = invoke_agent(access_token, "What was my previous question?", session_id1)
print(f"Session ID used: {session_id2}")
print(json.dumps(result2, indent=2))

間違ったスコープ名での未認可アクセスのスコープ検証をテストします

スコープはアプリケーションがどの権限を持つかを定義し、OAuth2 認可の細かいアクセス制御を提供します

In [ ]:
import jwt

def check_token_scopes(access_token, required_scope='agentcore'):
    """Check if token has required scope"""
    try:
        # Decode token without verification for demo (in production, verify signature)
        decoded = jwt.decode(access_token, options={"verify_signature": False})
        token_scopes = decoded.get('scp', [])
        
        # Check if token has required scope
        has_access = required_scope in token_scopes
        
        return {
            'has_access': has_access,
            'token_scopes': token_scopes,
            'required_scope': required_scope
        }
    except Exception as e:
        return {'error': str(e), 'has_access': False}

# Test 3: Scope validation - Negative scenario
print("=" * 50)
print("TEST 3: Scope Validation - Negative Scenario")
print("=" * 50)

# Check for wrong scope name - 'admin' scope would grant administrative privileges
wrong_scope_check = check_token_scopes(access_token, 'admin')
print(f"Wrong scope check result: {json.dumps(wrong_scope_check, indent=2)}")

if wrong_scope_check.get('has_access'):
    print("✅ Token has required scope")
else:
    print("❌ Access denied: Token does not have required scope")
    print(f"Token has scopes: {wrong_scope_check['token_scopes']}")
    print(f"Required scope: {wrong_scope_check['required_scope']}")
    print("Agent invocation would be blocked at application level")

正しいスコープ名での認可アクセスのスコープ検証をテストします

スコープは、認証されたアプリケーションが実行できるアクションを制限することで、安全な API アクセスを可能にします

In [ ]:
# Test 4: Scope validation - Positive scenario
print("=" * 50)
print("TEST 4: Scope Validation - Positive Scenario")
print("=" * 50)

# Check for correct scope - 'agentcore' scope grants access to invoke AgentCore agents
scope_check = check_token_scopes(access_token, 'agentcore')
print(f"Scope check result: {json.dumps(scope_check, indent=2)}")

if scope_check.get('has_access'):
    print("✅ Token has required scope - proceeding with agent call")
    result4, session_id4 = invoke_agent(access_token, "I have the right scope! Tell me about AWS security.")
    print(f"Agent response: {result4}")
else:
    print("❌ Token lacks required scope - access denied")

## まとめとクリーンアップ
このノートブックでは以下のことを学びました：
- OAuth 認証コードフローを提供するための Okta API とアプリケーションのセットアップ
- AgentCore Runtime を作成し、Okta を使用した Inbound 認証でエージェントをデプロイ
- トークンを取得して保護されたエージェントへのアクセスに使用
- セッション管理と継続性のデモンストレーション

#### 作成されたリソース

In [ ]:
# Display the created agent ID
if hasattr(launch_result, 'agent_id'):
    print(f"Agent ID: {launch_result.agent_id}")
    print(f"Agent ARN: {launch_result.agent_arn}")
else:
    print("Agent information not available")

#### AgentCore Runtime を削除

このチュートリアルで作成されたリソースをクリーンアップします：

In [ ]:
# Delete the AgentCore Runtime
try:
    if hasattr(launch_result, 'agent_id'):
        agentcore_control_client = boto3.client("bedrock-agentcore-control", region_name=region)
        agentcore_control_client.delete_agent_runtime(agentRuntimeId=launch_result.agent_id)
        print(f"✅ Agent {launch_result.agent_id} deleted successfully")
    else:
        print("⚠️  No agent ID found to delete")
except Exception as e:
    print(f"❌ Error deleting agent: {e}")
    print("You may need to delete the agent manually from the AWS console")

## まとめ

このノートブックでは以下のデモンストレーションを行いました：

1. **Okta IDP のセットアップ** - Okta テナント、アプリケーション、認可サーバーの設定
2. **シンプルなエージェントの作成** - セッション対応の基本的なエージェントの構築
3. **OAuth 認証の設定** - Okta JWT 検証を使用した AgentCore Runtime のセットアップ
4. **認証付きデプロイ** - Inbound 認証付きでエージェントをデプロイ
5. **認証フローのテスト** - OAuth トークンフローとセッション管理の検証

### 主な学び：

- **Inbound 認証**：Okta がエージェントエンドポイントを保護し、認証されたユーザーのみがエージェントを呼び出せることを保証
- **セッション管理**：エージェントはパーソナライズされた応答のためにセッション情報にアクセス可能
- **JWT トークン検証**：AgentCore は Okta JWT トークンを自動的に検証
- **セキュリティ**：認証されていないリクエストは自動的に拒否

### 次のステップ：

- ユーザーベースの認証フローの実装
- ユーザーコンテキストを持つより高度なエージェントロジックの追加
- 外部 API へのアクセスのための Outbound 認証の探索
- 追加のセキュリティレイヤーのための AgentCore Gateway との統合